라벨 데이터 구축을 위한 노트북
labeled_dir = r"D:\Landslide\data\processed\gyeongnam\label_join_with_SU.gpkg"
이 경로 파일의 속성에 date, cat이 저장되어있다. cat은 사면 고유번호를 의미한다.

샘플링 규칙:
기간은 20200310~20200930 사이에 발생한 샘플을 positive샘플로 선택한다. (현재 데이터 구축된 기간) - 약 340여개 샘플 

negative sample은 모델에게 '안정적인 사면'에 대한 정보를 주기 위한 것으로, 산사태가 발생하지 않았던 사면을 positive 샘플의 개수와 동일하게 선택하고, 각 사면의 date는 0511~0920 기간 사이로 랜덤하게 선택한다.
이때 참고 파일은 라벨링된 데이터가 아니라, "D:\Landslide\data\processed\gyeongnam\경남_전체기간_label_SU_join.gpkg" 이 경로에 있는 2011~2023 전체 산사태 발생 좌표 + 사면을 join한 데이터파일을 기준으로, 산사태가 발생하지 않은 사면 중에 샘플링한다. 


산출물은 cat	event_date	label 의 칼럼으로 구성하여 csv로 내보낸다. (label은 0,1 이진)

In [7]:
import geopandas as gpd
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# 파일 경로 설정
# 모든 산사태 사건-SU 조인 파일 (전체 기간)
landslide_join_path = r"D:\Landslide\data\processed\gyeongnam\gyeongnam_landslide_slope_join_v3.gpkg"
# 전체 사면 단위 파일
slope_units_path = r"D:\Landslide\data\processed\gyeongnam\SU\slope_properties_v3.gpkg"

In [8]:
# 1. Positive 샘플 추출 (2020-03-10 ~ 2020-09-30)
print("="*60)
print("STEP 1: Extracting Positive Samples")
print("="*60)

print("\nLoading landslide-slopeunit join data...")
landslide_gdf = gpd.read_file(landslide_join_path)

print(f"Total records: {len(landslide_gdf)}")
print(f"Columns: {landslide_gdf.columns.tolist()}")
print(f"\nFirst few rows:")
print(landslide_gdf.head())

# date 칼럼 확인
print(f"\nDate column info:")
print(f"  dtype: {landslide_gdf['date'].dtype}")
print(f"  NaN count: {landslide_gdf['date'].isna().sum()}")
print(f"  Sample values: {landslide_gdf['date'].head().tolist()}")

# cat 칼럼 확인
print(f"\nCat column info:")
print(f"  dtype: {landslide_gdf['cat'].dtype}")
print(f"  NaN count: {landslide_gdf['cat'].isna().sum()}")
print(f"  Unique cats: {landslide_gdf['cat'].nunique()}")

# date와 cat이 모두 유효한 레코드만 선택
valid_gdf = landslide_gdf[(landslide_gdf['date'].notna()) & (landslide_gdf['cat'].notna())].copy()
print(f"\nValid records (date & cat not null): {len(valid_gdf)}")

# date를 datetime으로 변환 (float 형식: 20200612.0 -> datetime)
valid_gdf['date'] = valid_gdf['date'].astype(int).astype(str)
valid_gdf['date'] = pd.to_datetime(valid_gdf['date'], format='%Y%m%d')

# 기간 필터링 (2020-03-10 ~ 2020-09-30)
start_date = pd.to_datetime('2011-01-01')
end_date = pd.to_datetime('2024-12-12')

positive_samples = valid_gdf[
    (valid_gdf['date'] >= start_date) & 
    (valid_gdf['date'] <= end_date)
].copy()

print(f"\n✓ Positive samples extracted: {len(positive_samples)}")
print(f"  Date range: {positive_samples['date'].min()} to {positive_samples['date'].max()}")
print(f"  Unique slope units: {positive_samples['cat'].nunique()}")

# Positive 샘플 데이터프레임 생성
positive_df = pd.DataFrame({
    'cat': positive_samples['cat'].astype(int),
    'event_date': positive_samples['date'].dt.strftime('%Y-%m-%d'),
    'label': 1
})

positive_output_path = r"D:\Landslide\data\model_ready\gyeongnam_positive_samples.csv"
positive_df.to_csv(positive_output_path,index=False, encoding='utf-8-sig')
print(f"\nPositive samples preview:")
print(positive_df.head(10))

STEP 1: Extracting Positive Samples

Loading landslide-slopeunit join data...
Total records: 1339
Columns: ['date', 'latitude', 'longitude', 'cat', 'area_m2', 'dem_average', 'slope_average', 'aspect_average', 'curv_plan_average', 'curv_prof_average', 'twi_average', 'lnspi_average', 'tri_sd3_average', 'accm_average', 'rock1_average', 'rock2_average', 'rock3_average', 'rock4_average', 'tpi90_average', 'has_forestroad', 'dist_fault', 'dist_stream', 'agri_average', 'bareland_average', 'forest_average', 'grass_average', 'urban_average', 'water_average', 'geometry']

First few rows:
         date   latitude   longitude    cat   area_m2  dem_average  \
0  20110709.0  35.070233  128.634900  72808  132300.0    31.607341   
1  20110709.0  35.175038  128.038560  63675   72000.0    71.260675   
2  20110709.0  35.142068  128.171511  66704  166500.0    45.703930   
3  20110709.0  35.150309  128.044733  66281  207900.0    93.424281   
4  20110709.0  35.114670  128.225066  69241   59400.0   120.328762

In [9]:
# 2. Negative 샘플 추출 (산사태가 발생하지 않은 사면)
print("\n" + "="*60)
print("STEP 2: Extracting Negative Samples")
print("="*60)

# 2-1. 산사태가 발생한 사면 목록 (cat이 유효한 것만, 전체 기간)
print("\nIdentifying landslide-affected slope units...")
landslide_cats = set(valid_gdf['cat'].unique())
print(f"Slope units with landslides (all periods): {len(landslide_cats)}")

# 2-2. 전체 사면 단위 데이터 로드
print(f"\nLoading complete slope units dataset...")
try:
    slope_units_gdf = gpd.read_file(slope_units_path)
    print(f"✓ Loaded from: {slope_units_path}")
    print(f"  Total slope units: {len(slope_units_gdf)}")
    
    # cat 칼럼 확인
    if 'cat' in slope_units_gdf.columns:
        all_cats = set(slope_units_gdf['cat'].unique())
    else:
        print("  Warning: 'cat' column not found. Available columns:", slope_units_gdf.columns.tolist())
        # cat 칼럼이 다른 이름일 경우 대체 (예: 'SU_ID', 'id' 등)
        raise Exception("cat column not found")
        
except Exception as e:
    print(f"⚠ Could not load slope units file: {e}")
    print(f"  Using all unique cats from landslide join data instead...")
    
    # landslide_gdf의 모든 고유 cat 사용 (산사태 유무 관계없이)
    all_cats = set(landslide_gdf['cat'].dropna().unique())
    print(f"  Total slope units (from landslide data): {len(all_cats)}")

# 2-3. 안정적인 사면 추출 (negative 샘플 풀)
safe_cats = list(all_cats - landslide_cats)
print(f"\n✓ Safe slope units (no landslides): {len(safe_cats)}")

# 2-4. Negative 샘플링
num_positive = len(positive_df)
print(f"\nSampling {num_positive} negative samples...")

if len(safe_cats) < num_positive:
    print(f"⚠ Warning: Not enough safe slopes ({len(safe_cats)}) for {num_positive} samples")
    print(f"  Using all {len(safe_cats)} available safe slopes")
    sampled_cats = safe_cats
else:
    np.random.seed(42)
    sampled_cats = np.random.choice(safe_cats, size=num_positive, replace=False)
    print(f"✓ Sampled {len(sampled_cats)} safe slope units")

# 2-5. 랜덤 날짜 부여 (2020-05-11 ~ 2020-09-20)
print(f"\nAssigning random dates (2020-05-11 to 2020-09-20)...")
date_start = datetime(2020, 5, 11)
date_end = datetime(2020, 9, 20)
date_range_days = (date_end - date_start).days

np.random.seed(42)
random_dates = []
for _ in sampled_cats:
    random_day = np.random.randint(0, date_range_days + 1)
    random_date = date_start + timedelta(days=random_day)
    random_dates.append(random_date.strftime('%Y-%m-%d'))

# 2-6. Negative 샘플 데이터프레임 생성
negative_df = pd.DataFrame({
    'cat': [int(c) for c in sampled_cats],
    'event_date': random_dates,
    'label': 0
})

print(f"\n✓ Negative samples created: {len(negative_df)}")
print(f"  Date range: {negative_df['event_date'].min()} to {negative_df['event_date'].max()}")

print(f"\nNegative samples preview:")
print(negative_df.head(10))


STEP 2: Extracting Negative Samples

Identifying landslide-affected slope units...
Slope units with landslides (all periods): 1017

Loading complete slope units dataset...
✓ Loaded from: D:\Landslide\data\processed\gyeongnam\SU\slope_properties_v3.gpkg
  Total slope units: 89529

✓ Safe slope units (no landslides): 88512

Sampling 1325 negative samples...
✓ Sampled 1325 safe slope units

Assigning random dates (2020-05-11 to 2020-09-20)...

✓ Negative samples created: 1325
  Date range: 2020-05-11 to 2020-09-20

Negative samples preview:
     cat  event_date  label
0  47717  2020-08-21      0
1  81233  2020-08-11      0
2  16297  2020-05-25      0
3  70231  2020-08-25      0
4  76623  2020-07-21      0
5  88233  2020-05-31      0
6  60483  2020-08-21      0
7  30420  2020-09-09      0
8  66481  2020-07-24      0
9  48186  2020-08-06      0


In [10]:
# 3. Positive와 Negative 샘플 결합 및 CSV 출력
print("\n=== Final Dataset ===")

# 데이터프레임 결합
final_df = pd.concat([positive_df, negative_df], ignore_index=True)

# 정렬 (선택사항: cat 또는 event_date로 정렬)
final_df = final_df.sort_values(by='event_date').reset_index(drop=True)

print(f"Total samples: {len(final_df)}")
print(f"Positive samples: {(final_df['label'] == 1).sum()}")
print(f"Negative samples: {(final_df['label'] == 0).sum()}")
print(f"\nFirst 10 rows:")
print(final_df.head(10))
print(f"\nLast 10 rows:")
print(final_df.tail(10))

# CSV 파일로 저장
output_path = r"D:\Landslide\data\processed\gyeongnam\landslide_samples.csv"
final_df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"\n✓ Dataset saved to: {output_path}")

# 기본 통계
print(f"\n=== Dataset Statistics ===")
print(f"Date range: {final_df['event_date'].min()} to {final_df['event_date'].max()}")
print(f"Unique slope units (cat): {final_df['cat'].nunique()}")
print(f"Class balance: {(final_df['label'] == 1).sum()}/{(final_df['label'] == 0).sum()} (positive/negative)")


=== Final Dataset ===
Total samples: 2650
Positive samples: 1325
Negative samples: 1325

First 10 rows:
     cat  event_date  label
0  72808  2011-07-09      1
1  25329  2011-07-09      1
2  19996  2011-07-09      1
3  12883  2011-07-09      1
4  18764  2011-07-09      1
5  24518  2011-07-09      1
6  41840  2011-07-09      1
7  43622  2011-07-09      1
8  23562  2011-07-09      1
9  18953  2011-07-09      1

Last 10 rows:
        cat  event_date  label
2640  86639  2023-07-16      1
2641  26762  2023-07-17      1
2642   7178  2023-07-18      1
2643   2046  2023-07-18      1
2644   3891  2023-07-18      1
2645   4501  2023-07-18      1
2646   2222  2023-07-18      1
2647  11097  2023-07-18      1
2648   6834  2023-07-18      1
2649  60289  2023-08-10      1

✓ Dataset saved to: D:\Landslide\data\processed\gyeongnam\landslide_samples.csv

=== Dataset Statistics ===
Date range: 2011-07-09 to 2023-08-10
Unique slope units (cat): 2342
Class balance: 1325/1325 (positive/negative)
